学习[scqubits](https://github.com/scqubits/scqubits)有关 写电路自动量子化的程序 的部分 主要是学习scqubits/core/symbolic_circuit_graph.py与scqubits/core/symbolic_circuit_graph.py中的部分代码


## Ai建议的学习路径

这是一个非常棒的学习计划。`scqubits` 的核心正是基于你上传的那篇文档（通常对应于 **Ulrich et al., 2016** 或类似的 **Min-GKP** 理论）所描述的图论量子化方法。

要将书本上的公式（你上传的图片）与代码（`scqubits` 源码）对应起来，我建议采取 **“从数据结构到物理流程”** 的四步走学习姿势。

以下是具体的代码阅读指南，帮助你把数学公式映射到 Python 实现上：



### 第一阶段：图的构建与生成树 (Graph Topology)



**对应理论：** 图片中的 "Label and order branches", "Spanning tree", "Twigs and Links".

**核心代码文件：** `scqubits/core/symbolic_circuit_graph.py`

1. **节点与支路 (Nodes & Branches):**
* **理论**：电路被视为图 ，由节点  和支路  组成。
* **代码**：查看 `class Node` 和 `class Branch`。
* **重点**：注意 `Branch` 类中如何存储 `nodes`（连接点）和 `parameters`（）。


2. **生成树 (Spanning Tree):**
* **理论**：选择树枝（twigs）构成生成树，剩下的为连支（links）。这是定义  和  矩阵的基础。
* **代码**：重点阅读 `SymbolicCircuitGraph._spanning_tree` 方法。
* **核心逻辑**：
* 代码并没有仅仅随机选一个树，它有一个物理上的优先级：它会**移除电容支路**来构建生成树（参见代码注释 `removing all the capacitive branches`）。这是为了处理超导电路中常见的“悬空节点”或“电容子图”问题，比纯数学的图论更贴近物理。
* 返回值包含 `list_of_trees` 和 `closure_branches_for_trees`（这实际上就是对应的连支 Links）。







### 第二阶段：变量变换与自由度归类 (Variable Transformation)



**对应理论：** 图片中的 "c. Reducing the number of coordinates", .

这是代码中最复杂也是最精彩的部分。`scqubits` 不仅仅是简单地划分树枝/连支，它还自动识别了**变量类型**（周期性的、延伸的、冻结的等）。

**核心代码文件：** `scqubits/core/symbolic_circuit_graph.py`

1. **变量分类 (Mode Classification):**
* **理论**：通过  矩阵将连支磁通  用树枝磁通  表示。
* **代码**：阅读 `SymbolicCircuitGraph.check_transformation_matrix` 和 `variable_transformation_matrix`。
* **关键点**：
* 代码并没有直接显式地构建纯粹的  矩阵然后乘一下，而是通过寻找**零空间 (Null Space)** 和线性无关组来找到变换矩阵。
* `_independent_modes` 方法：这是核心算法。它寻找独立的回路。
* `var_categories`：代码将变量分为 `periodic` (对应环路中的通量，如 Fluxonium), `extended` (对应开放线路，如 Transmon), `frozen` (无动力学项), `free` (自由电荷)。这比文档中的纯矩阵推导更进了一步，处理了简并情况。




2. **构建变换矩阵 (Transformation Matrix):**
* 代码最终会生成一个 `transformation_matrix`。这个矩阵的作用等同于你文档中公式 (B10) 的  的推广形式，用于将节点磁通  映射到新的广义坐标 。





### 第三阶段：构建物理矩阵 (M & K Matrices)



**对应理论：** 图片中的 Eq (B14) , Eq (B15) , 以及最终的 Eq (B22)  (Mass) 和  (Spring)。

**核心代码文件：** `scqubits/core/symbolic_circuit.py`

1. **电容与电感矩阵 (C & L Matrices):**
* **理论**： 是电容对角阵， 是电感矩阵。
* **代码**：
* `SymbolicCircuit._capacitance_matrix()`：构建节点电容矩阵。
* `SymbolicCircuit._inductance_inverse_matrix()`：构建节点电感逆矩阵。
* **观察**：代码直接在节点基（Node basis）下构建这些矩阵，因为这比直接在混合基下构建更容易，然后再应用变换矩阵。




2. **变换到新坐标系:**
* **理论**：公式 (B22a) 。
* **代码**：
* 在 `_EC_matrix` 方法中，你会看到代码执行了 `transformation_matrix.T @ ... @ transformation_matrix` 的操作。这就是在执行坐标变换，将节点电容矩阵变换为广义坐标下的“质量”矩阵（这里实际上求的是 ，即逆电容矩阵）。







### 第四阶段：符号化拉格朗日量与哈密顿量 (Lagrangian & Hamiltonian)



**对应理论：** 图片中的 "2. Equations of motion", Eq (B21).

**核心代码文件：** `scqubits/core/symbolic_circuit.py`

1. **生成拉格朗日量:**
* **代码**：`SymbolicCircuit.generate_symbolic_lagrangian`。
* **逻辑**：
1. 定义符号变量  和  (利用 `sympy`)。
2. 计算动能项 `C_terms` ()。
3. 计算势能项 `potential` (电感能 + 约瑟夫森结能)。
4. **关键差异**：文档中推导的是微分方程 (Motion Equation)，而代码直接构建标量函数 。因为有了 ，利用 Legendre 变换 () 得到哈密顿量在计算机代数系统中更容易实现。




2. **生成哈密顿量:**
* **代码**：`SymbolicCircuit.generate_symbolic_hamiltonian`。
* **逻辑**：它使用了之前计算的 `_EC_matrix` (充电能矩阵) 来构建动能项 ，并加上势能项。





### 建议的练习步骤 (The "Pose")



不要试图一次看懂所有代码。请按照以下顺序“打断点”或阅读：

1. **Step 1: 手算 vs 代码运行**
* 找一个最简单的电路（例如 Transmon：1个C，1个JJ，并联）。
* 用手按照文档推导  矩阵。
* 运行 `scqubits` 的 `SymbolicCircuit`，打印 `circuit.transformation_matrix` 和 `circuit.var_categories`。看看代码自动找到的变量和你的手算结果是否一致。


2. **Step 2: 追踪 `configure()**`
* `scqubits/core/symbolic_circuit.py` 中的 `configure` 方法是总指挥。
* 它是所有初始化的入口。从这里开始阅读，看它依次调用了 `variable_transformation_matrix` -> `generate_symbolic_lagrangian` -> `generate_symbolic_hamiltonian`。


3. **Step 3: 关注 `sympy` 的使用**
* 注意代码是如何大量使用 `sympy` 进行符号运算的。这对于“自动量子化”至关重要，因为我们需要的是解析表达式（Symbolic Hamiltonian），而不仅仅是数值。



**总结一句话：**
你的文档是在讲**物理原理（基于割集的运动方程）**，而 `scqubits` 的代码是在实现**基于该原理的符号计算引擎**。最关键的桥梁是 `transformation_matrix`，它对应了你文档中的矩阵变换操作。

## 图的构建与生成树

### 节点与枝

#### 节点

这个代码来自scqubits\scqubits\core\symbolic_circuit_graph.py的39行
使用##在代码中做笔记

修改了类型的声明方式

In [26]:

import copy
from sympy import symbols, Symbol

#from typing import Any, Dict, List, Optional, Set, Tuple, Union
##作用：它们告诉阅读代码的人（以及 IDE 编辑器），这个变量应该装什么类型的数据。
##Dict[str, int]：表示这是一个字典，键（Key）是字符串，值（Value）是整数。
##List[Branch]：表示这是一个列表，里面装的每一个元素都必须是 Branch 类的实例。

##但是新版Py不需要这个

## scqubits\scqubits\core\symbolic_circuit_graph.py line 39

class Node:
    """Class representing a circuit node, and handled by `Circuit`. The attribute
    `branches` is a list of `Branch` objects containing all branches connected to the
    node.

    Parameters
    ----------
    id: int
        integer identifier of the node
    marker: int
        An internal attribute used to group nodes and identify sub-circuits in the
        method independent_modes.
    """

    def __init__(self, index: int, marker: int = 0):
        self.index: int = index
        self.marker: int = marker
        self._init_params: dict[str, int] = {"id": self.index, "marker": self.marker}
        self.branches: list[Branch] = []

##这两行__str__和__repr__(只是让你打印一个节点对象是可以得到"Node 编号",  而非<__main__.Node object at 0x7f8b1c2d3a90>
    def __str__(self) -> str:
        return "Node {}".format(self.index)

    def __repr__(self) -> str:
        return "Node({})".format(self.index)


    def connected_nodes(self, branch_type: str) -> list["Node"]:
        """Returns a list of all nodes directly connected by branches to the current
        node, either considering all branches or a specified `branch_type` - ("C", "L", "JJ", "all") for capacitive, inductive, Josephson junction, or all types of branches.
        """
        result = []
        if branch_type == "all":
            branch_list = self.branches
        else:
            branch_list = [
                branch for branch in self.branches if branch.type == branch_type
            ]
        for branch in branch_list:
            if branch.nodes[0].index == self.index:
                result.append(branch.nodes[1])
            else:
                result.append(branch.nodes[0])
        return result

    def is_ground(self) -> bool:
        """Returns a bool if the node is a ground node (Node with index set to 0)."""
        return True if self.index == 0 else False

    def __deepcopy__(self, memo):
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        for k, v in self.__dict__.items():
            setattr(result, k, copy.deepcopy(v, memo))
        return result

#### 枝

In [5]:

## scqubits\scqubits\core\circuit_utils.py line 36
def _junction_order(branch_type: str) -> int:
    """Returns the order of the branch if it is a JJ branch, if the order is n its
    energy is given by cos(phi) + cos(2*phi) + ... + cos(n*phi)

    Args:
        branch (_type_): Branch

    Raises:
        ValueError: when the branch is not a josephson junction

    Returns
        _type_: int, order of the josephson junction
    """
    if "JJ" not in branch_type:
        raise ValueError("The branch is not a JJ branch")
    if len(branch_type) > 2:
        if (
            branch_type[2] == "s"
        ):  # adding "JJs" which is a junction with sawtooth current phase relationship
            return 1
        return int(branch_type[2:])
    else:
        return 1



|**元件类型 (branch_type)**|**列表内容 (parameters)**|**列表长度**|**示例代码**|
|---|---|---|---|
|**"C"** (电容)|`[电容能]`|1|`[1.0]`|
|**"L"** (电感)|`[电感能]`|1|`[0.5]`|
|**"JJ"** (约瑟夫森结)|`[EJ, ECJ]`|2|`[20.0, 0.5]`|
|**"JJ2"** (二阶结)|`[EJ1, EJ2, ECJ]`|3|`[10.0, 0.1, 0.5]`|



传入 `__init__` 的**参数** (`parameters`) 是一个 `list`，但是最终存储在对象身上的**属性** (`self.parameters`) 被转换成了一个 `dict`。
```Python
def __init__(self, ..., parameters: list, ...):
    # 此时，parameters 是一个列表，例如 [10, 0.2]
    # self.parameters 这个属性还不存在！
    
    self._set_parameters(parameters) # 把列表传给处理函数
```

**第二步：在 `_set_parameters` 中加工并赋值** 这里是“魔术”发生的地方。这个函数接收了列表，把它拆开，加上标签（Key），然后赋给了 `self.parameters`。



```Python
def _set_parameters(self, parameters) -> None:
    # 这里的 parameters 还是那个列表 [10, 0.2]
    
    if "JJ" in self.type:
        # ... 逻辑判断 ...
        
        # 关键时刻！！！
        # 这里创建了一个空字典，并赋值给了 self.parameters
        self.parameters = {} 
        
        # 然后把列表里的值取出来，填进字典里
        self.parameters["EJ"] = parameters[0]   # 10
        self.parameters["ECJ"] = parameters[1]  # 0.2
```



In [6]:


## scqubits\scqubits\core\symbolic_circuit_graph.py line 96

class Branch:
    """Class describing a circuit branch, used in the Circuit class.

    Parameters
    ----------
    n_i:
        initial `Node` of the branch
    n_f:
        final `Node` of the branch
    branch_type:
        is the type of this Branch, example `"C"`, `"JJ"` or `"L"`
    parameters:
        list of parameters for the branch, namely for
        capacitance: `[EC]`;
        for inductance: `[10]`;
        for Josephson Junction: `[EJ, 1]`
    aux_params:
        Dictionary of auxiliary parameters which map a symbol from the input file a numeric parameter.

    Examples
    --------
    `Branch("C", Node(1, 0), Node(2, 0))` is a capacitive branch connecting the nodes with indices 0 and 1.
    """

    def __init__(
        self,
        n_i: Node,
        n_f: Node,
        branch_type: str,
        parameters: list[float| Symbol | int],
        index: int|None = None,
        aux_params: dict[Symbol, float] = {},
    ):
        self.nodes = (n_i, n_f)
        self.type = branch_type
        self.index = index
        # store info of current branch inside the provided nodes
        # setting the parameters if it is provided
        self._set_parameters(parameters)
        self.aux_params = aux_params

        ## 这使得Node类可以访问self.branches, 建立了Node之间的双向链接
        self.nodes[0].branches.append(self)
        self.nodes[1].branches.append(self)

    def __str__(self) -> str:
        return (
            "Branch "
            + self.type
            + " connecting nodes: ("
            + str(self.nodes[0].index)
            + ","
            + str(self.nodes[1].index)
            + "); "
            + str(self.parameters)
        )

    def __repr__(self) -> str:
        return f"Branch({self.type}, {self.nodes[0].index}, {self.nodes[1].index}, index: {self.index})"

## 接受传入的参数, 如果支是C/L就直接改变键E
## 如果含有JJ, 就先判断阶数,  
    def _set_parameters(self, parameters) -> None:
        if self.type in ["C", "L"]:
            self.parameters = {f"E{self.type}": parameters[0]}
            ##重新赋为一个字典了
        elif "JJ" in self.type:
            number_of_junc_params = _junction_order(self.type)
            self.parameters = {}
            for junc_order in range(1, number_of_junc_params + 1):
                if junc_order == 1:
                    self.parameters["EJ"] = parameters[0]
                else:
                    self.parameters[f"EJ{junc_order}"] = parameters[junc_order - 1]
            self.parameters["ECJ"] = parameters[number_of_junc_params]

    def node_ids(self) -> tuple[int, int]:
        """Returns the indices of the nodes connected by the branch."""
        return self.nodes[0].index, self.nodes[1].index

    def is_connected(self, branch) -> bool:
        """Returns a boolean indicating whether the current branch is connected to the
        given `branch`"""
        distinct_node_count = len(set(self.nodes + branch.nodes))
        if distinct_node_count < 4:
            return True
        return False

    def common_node(self, branch) -> set[Node]:
        """Returns the common nodes between self and the `branch` given as input."""
        return set(self.nodes) & set(branch.nodes)

    def __deepcopy__(self, memo):
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        for k, v in self.__dict__.items():
            setattr(result, k, copy.deepcopy(v, memo))
        return result

#### 简化枝

高阶JJ超出了我们的需要, 但现在暂时留着吧

### 生成树

```Python
class SymbolicCircuitGraph(ABC):
```
是一个抽象基类, 隔离出来一些方法, 暂时不管这个类, 直接看最关键的
```Python
   def _spanning_tree(
        self, consider_capacitive_loops: bool = False, use_closure_branches: bool = True
    ):
```

来自`scqubits.utils.misc` :

In [24]:
def flatten_list_recursive(some_list: list) -> list:
    """Flattens a list of lists recursively.

    Parameters
    ----------
    some_list:
        A list of lists, which can hold any class instance.

    Returns
    -------
    Flattened list of objects
    """
    if some_list == []:
        return some_list
    if isinstance(some_list[0], list):
        return flatten_list_recursive(some_list[0]) + flatten_list_recursive(
            some_list[1:]
        )
    return some_list[:1] + flatten_list_recursive(some_list[1:])


def unique_elements_in_list(list_object: list) -> list:
    """Returns a list of all the unique elements in the list.

    Parameters
    ----------
    list_object :
        A list of any objects
    """
    unique_list = []
    [
        unique_list.append(element)
        for element in list_object
        if element not in unique_list
    ]
    return unique_list
